In [1]:
import os
import time
import numpy as np
import pandas as pd
from gensim import corpora, utils
from gensim.models import LdaMulticore
from gensim.models.wrappers import DtmModel

In [14]:
data = pd.read_json(
    "../data/raw/210112 - evp_integrum.json",
    orient="records",
    convert_dates=False
)

In [15]:
data["date"] = pd.to_datetime(data["date"])
data = data.loc[data["date"].notnull() & data["fulltext"].notnull()].copy()
data = data.loc[data["date"] <= "2021-01-01"].copy()
data = data.sort_values("date").reset_index(drop=True)
data.shape

(13062, 21)

In [16]:
#now we count the number of full texts in each years
dates_count = data.groupby(pd.Grouper(key="date", freq="Y")).agg(n_paragraphs = ("fulltext", "size"))
dates_count = dates_count.loc[dates_count['n_paragraphs'].ne(0)].reset_index()
dates_count["n_paragraphs"].sum()

13062

In [17]:
time_seq = dates_count["n_paragraphs"].to_list()
sum(time_seq)

13062

# Dictionary and BOWs

In [18]:
# create a dictionary
dictionary = corpora.Dictionary(data['fulltext'].str.split())  
print(f'{len(dictionary)} tokens overall') 

803219 tokens overall


In [21]:
data['bows'] = data['fulltext'].str.split().apply(dictionary.doc2bow)  # convert documents (list of tokens) to BOWs
# print(data['bows'][0]) #what we have

In [24]:
start = time.time()
model = DtmModel(
    dtm_path="../models/main",
    corpus=data["bows"],
    time_slices=time_seq,
    num_topics=20,
    id2word=dictionary,
    initialize_lda=True,
    top_chain_var=0.05
)
finish = time.time()

OSError: [WinError 193] %1 is not a valid Win32 application

In [ ]:
finish - start